This is just 1 plate per experiment, 3 different dates.

ABCD etc are different.

Column 1 2 3 = TS121 
Column 4 5 6 = TS163 

Column 7 8 9 = TS121 
Column 10 11 12 = TS163 

For example:

A1 A2 A3 are the same

No drugs in the plates.

Plate content (Black Flat BioGreiner Plate)

Col 1-6 -Row A - H:
0, 1E2, 1E3, 1E4, 1E5, 1E6, 1E7, 1E8 nM Arabinose

Col 7-12 - Row A - H: 
0, 1.5E4, 3E4, 6E4, 1.25E5, 2.5E5, 5E5, 1E6 nM Arabinose
 

Files
All files represent the same biological experiment

NB_OD600_mEGFP = old format 
R98 & R97 = new format that will be used in the future

In [1]:
old_files = ['./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92.csv']
new_files = ['./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1.csv','./dataset/R98 version 2 - SparkControl magellan Sheet 1.csv']

In [2]:
all_files = new_files[:]
all_files.extend(old_files)
print all_files

['./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1.csv', './dataset/R98 version 2 - SparkControl magellan Sheet 1.csv', './dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92.csv']


In [3]:
#fn = new_files[1]
def replaceList(a, l, c):
    for n, i in enumerate(a):
        if i == l:
            a[n] = c            
    return a

# for a list of string
def removeS(l):
    l_out = []
    for s in l:        
        l_out.append(s.replace('s',''))
    return l_out

def read_data_from_new_format(fn):
    import csv
    import numpy as np
    with open(fn) as csvfile:
        csvReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        count = 0
        data = dict()
        m_od = []
        m_gfp = []
        time_gfp = []
        time_od = []
        c = 0
        for row in csvReader:
            if count == 0:
                #print row
                pos = row[1].split(',')
                pos_od = pos.index('OD600')
                pos_gfp50 = pos.index('GFP50')
                try:
                    pos_gfp = pos.index('GFP100')
                except:
                    print "GFP100 marker not found"
                    pos_gfp = pos_gfp50
                
            if count == 1:
                r =  row[0]
                l_t_od = r.split(',')[pos_od:pos_gfp]
                l_t_od = removeS(l_t_od)
                time_od.extend(map(int, l_t_od))
                l_t_gfp = r.split(',')[pos_gfp50:len(r.split(','))]
                l_t_gfp = removeS(l_t_gfp)
                replaceList(l_t_gfp,'','-1')
                time_gfp.extend(map(int,l_t_gfp))
            if count == 2:
                print "ignore temperature row"
            if count > 2:
                r =  row[0]
                row_list = r.split(',')
                row_list = replaceList(row_list,'','-1')
                row_list = replaceList(row_list,'Overflow','-1')
                row_list = replaceList(row_list,'Overflow','-1')
                m_od.append(map(float,row_list[pos_od:pos_gfp]))
                m_gfp.append(map(float,row_list[pos_gfp50:len(r.split(','))-1]))
                c = c + 1
                if r.split(',')[0] == 'H12':
                    break
            count = count + 1
        m = min(len(m_od[0]),len(m_gfp[0]))
        data['od'] = np.asmatrix(m_od[0:m]).transpose()
        data['gfp'] = np.asmatrix(m_gfp[0:m]).transpose()
        data['time_od'] = np.asmatrix(time_od[0:m]).transpose()
        data['time_gfp'] = np.asmatrix(time_gfp[0:m]).transpose()
    return data


all_data = dict()
for fn in new_files:
    print fn
    all_data[fn] = read_data_from_new_format(fn)
    print all_data[fn]['od'].shape
    print all_data[fn]['gfp'].shape

./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1.csv
ignore temperature row
(258, 96)
(258, 96)
./dataset/R98 version 2 - SparkControl magellan Sheet 1.csv
GFP100 marker not found
ignore temperature row
(568, 96)
(568, 96)


In [4]:
import numpy as np
m_value = []
fn=old_files[0]
import csv
li = '1,2,3,4,5,6,7,8,9,10,11,12'
c_time = 'Time'
t = []
n_row = 8
line_num = 0
find_start = False
with open(fn) as csvfile:
    csvReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in csvReader:           
        if li in row[0]:
            find_start = True
            line_num = csvReader.line_num
            val=[]
            #print row,csvReader.line_num
        if find_start and csvReader.line_num - line_num > 0 and csvReader.line_num - line_num <= n_row:
            lst = row[0].split(',')[1:]
            replaceList(lst, '','-1')
            replaceList(lst, 'OVER','-1')
            x = np.array(lst)
            y = x.astype(np.float)
            val.extend(y)            
        if find_start and csvReader.line_num - line_num > n_row:
            find_start = False
            m_value.append(val)
            #print row        
        if c_time in row[0] and len(row)>1:
            #print row[1].split(',')[1]
            t.append(row[1].split(',')[1])


m_ = np.asmatrix(m_value)

t = np.array(t)
t = t.astype(np.float)
#t = t/3600
t_ = np.asmatrix(t)
t_ = t_.transpose()
t_.shape


sec = []
for i, v in enumerate(t):
    if v==0.:
        sec.append(i)
sec.append(len(t))

m_OD = []
t_OD = []
m_GFPs = []
t_GFPs = []

for i in range(len(sec)-1):
    if i==0:
        m_OD=m_[sec[i]:sec[i+1],:]
        t_OD = t_[sec[i]:sec[i+1],:]
    else:
        m_GFPs.append(m_[sec[i]:sec[i+1],:])
        t_GFPs.append(t_[sec[i]:sec[i+1],:])
        
m_OD = m_OD[0:len(m_OD)-1,:]
t_OD = t_OD[0:len(t_OD)-1,:]
m_GFP = m_GFPs[2]
t_GFP = t_GFPs[2]

all_data[fn] = dict()
all_data[fn]['od'] = m_OD
all_data[fn]['gfp'] = m_GFP
all_data[fn]['time_od'] = t_OD
all_data[fn]['time_gfp'] = t_GFP

In [5]:
for k in all_data:
    print k
    print all_data[k]['od'].shape
    print all_data[k]['gfp'].shape
    print all_data[k]['time_od'].shape

./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1.csv
(258, 96)
(258, 96)
(258, 1)
./dataset/R98 version 2 - SparkControl magellan Sheet 1.csv
(568, 96)
(568, 96)
(568, 1)
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92.csv
(227, 96)
(227, 96)
(227, 1)


In [6]:
def smooth(a,WSZ=5):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)

    start = np.cumsum(a[:WSZ-1])[::2]/r
    #print start,out0
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

def digitalOnly(s):
    import string
    al=string.maketrans('','')
    nodigs=al.translate(al, string.digits)
    return s.translate(al, nodigs)

def getLinearRegressionCoef(X,y):
    import numpy as np
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    reg.score(X, y)    
    return reg.coef_


def computeDerivetive_od(t, sOD, wsz = 5):
    from numpy import isnan
    coef_OD = np.zeros(len(sOD))
    t_mean = np.zeros(len(sOD))
    j = 0
    nan_found = False
    for i in range(len(sOD)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]        
        sOD_wsz = np.expand_dims(np.log(sOD[i-wsz/2:i+wsz/2+1]),axis=1)    
        if len(sOD_wsz)!= wsz:
            coef_OD[i] = 0
        else:                
            if len(t_wsz)==len(sOD_wsz):                
                where_are_NaNs = isnan(sOD_wsz)
                if any(where_are_NaNs):
                    sOD_wsz[where_are_NaNs] = np.mean(sOD_wsz[~where_are_NaNs])
                    print "nan Found in array, replaced by mean value of the rest"

                coef_OD[i]=getLinearRegressionCoef(t_wsz,sOD_wsz)
                t_mean[i] = np.mean(t_wsz)
    
    dt = t_mean[wsz/2:len(t_mean)-wsz/2]
    dt_pad = np.pad(dt,[wsz/2,wsz/2],'reflect',reflect_type='odd')
    
    return coef_OD, dt_pad


def computeDerivetive(t, sOD, sGFP, wsz = 5):# od is firstly log computed
    from numpy import isnan
    coef_OD = np.zeros(len(sOD))
    t_mean = np.zeros(len(sOD))
    j = 0
    nan_found = False
    for i in range(len(sOD)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]        
        sOD_wsz = np.expand_dims(np.log(sOD[i-wsz/2:i+wsz/2+1]),axis=1)    
        if len(sOD_wsz)!= wsz:
            coef_OD[i] = 0
        else:                
            if len(t_wsz)==len(sOD_wsz):                    
                where_are_NaNs = isnan(sOD_wsz)
                if any(where_are_NaNs):
                    sOD_wsz[where_are_NaNs] = np.mean(sOD_wsz[~where_are_NaNs])
                    print "nan Found in array, replaced by mean value of the rest"
                coef_OD[i]=getLinearRegressionCoef(t_wsz,sOD_wsz)
                t_mean[i] = np.mean(t_wsz)

    coef_GFP = np.zeros(len(sGFP))
    for i in range(len(sGFP)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]
        sGFP_wsz = np.expand_dims(sGFP[i-wsz/2:i+wsz/2+1],axis=1)
        if len(sGFP_wsz)!= wsz:
            coef_GFP[i] = 0
        else:                
            if len(t_wsz)==len(sGFP_wsz):
                coef_GFP[i]=getLinearRegressionCoef(t_wsz,sGFP_wsz)

    dGFP = coef_GFP
    dOD = coef_OD
    dt = t_mean[wsz/2:len(t_mean)-wsz/2]
    dt_pad = np.pad(dt,[wsz/2,wsz/2],'reflect',reflect_type='odd')
    dGFPdt_divide_OD = dGFP/sOD
    return dGFPdt_divide_OD, coef_OD, coef_GFP, dt_pad

def draw_tangent(x1,y1,ind,e=0.1,plot=False):
    from scipy import interpolate
    import numpy as np
    import matplotlib.pyplot as plt
    x = np.squeeze(x1)
    y = np.squeeze(y1)
    # interpolate the data with a spline
    spl = interpolate.splrep(x,y)
    a = x[ind]
    fa = interpolate.splev(a,spl,der=0)     # f(a)
    fprime = interpolate.splev(a,spl,der=1) # f'(a)
    left = 0
    for i in range(ind,0,-1):        
        a_i = x[i]
        y_i = y[i]        
        tan_i = fa+fprime*(a_i-a) # tangent                
        if abs(tan_i-y_i) > abs(y[ind]*e):
            left = i
            break    
    right = len(x) - 1
    for i in range(ind+1,len(x)):        
        a_i = x[i]
        y_i = y[i]
        tan_i = fa+fprime*(a_i-a) # tangent        
        if abs(tan_i-y_i) > abs(y[ind]*e):
            right = i
            break
    small_t = np.linspace(x[left],x[right],10)
    tan = fa+fprime*(small_t-a) # tangent    
    if plot:
        plt.plot(a,fa,'om',small_t,tan,'--r')
        plt.plot(x,y)
    return left, right,small_t,tan


def getTimepoints_old(coef_OD,sOD_log, dt_pad,start_ind, thres=0.03, thres_range=0.8):
    coef_OD_fromStart_ind = coef_OD[start_ind:]
    ind = np.argmax(coef_OD_fromStart_ind) + start_ind        
    max_v = coef_OD[ind]
    
    time_max_coefod = dt_pad[ind]        
    right_side_exp = len(coef_OD)-1
    left_side_exp = 0
    for k in range(ind+1,len(coef_OD)):
        v_od = coef_OD[k]
        if (max_v-v_od)/max_v > 1-thres_range:
            right_side_exp = k
            break

    for k in reversed(range(ind)):
        v_od = coef_OD[k]
        if (max_v-v_od)/max_v > 1-thres_range:
            left_side_exp = k
            break

    time_left = dt_pad[left_side_exp]
    time_right = dt_pad[right_side_exp]  
    
    return time_max_coefod,time_left,time_right,left_side_exp,right_side_exp

def getTimepoints(coef_OD,sOD_log,dt_pad,start_ind):
    coef_OD_fromStart_ind = coef_OD[start_ind:]
    ind = np.argmax(coef_OD_fromStart_ind) + start_ind        
    max_v = coef_OD[ind]
    time_max_coefod = dt_pad[ind]        
    left_side_exp, right_side_exp,small_t,tan = draw_tangent(dt_pad,sOD_log,ind)
    time_left = dt_pad[left_side_exp]
    time_right = dt_pad[right_side_exp]  
    return time_max_coefod,time_left,time_right,left_side_exp,right_side_exp,small_t,tan

def generateWellName():
    list1 = map(chr, range(65, 73))
    list2 = range(1,13,1)
    list2 = map(str, list2)  
    well_name = []
    for l in list1:
        for n in list2:
            well_name.append(l+n)
    return well_name

well_name = generateWellName()
print well_name


def getPostiveStart(sOD):
    start_positive = 0
    for o in range(start_positive, len(sOD)):
        if sOD[o] <= 0:
            start_positive = o + 1
    if start_positive >= len(sOD):
        start_positive = -1
    return start_positive

def mkdir_p(path):
    import errno    
    import os
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
            
def getStationaryTime(sOD):
    sOD = np.asarray(sOD)
    sta_index = len(sOD)-1
    last_value = sOD[sta_index]
    for i in range(len(sOD)-2,0,-1):
        if (abs(last_value-sOD[i])/last_value)>0.05:
            sta_index = i
            print "stationary phase found"
            break
    return sta_index

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12']


In [7]:
def getStats(m_OD, time,m_GFP, outFolder):
    import numpy as np
    import os
    preName = os.path.split(outFolder)[1]
    outName_Stat = os.path.join(outFolder, preName +'_stats.csv')
    figure_folder_path = os.path.join(outFolder,'FIG')
    mkdir_p(figure_folder_path)
    
    detectedColumn = range(96)
    names_array = map(str,range(96))
    filtered_name = []
    t = time.reshape(-1,1)
    t_k = (t[1:len(t)] - t[0:len(t)-1])/3600.0
    t_k = np.cumsum(t_k)
    t_k = np.insert(t_k,0,0)
    t_k = t_k.reshape(-1,1)
    
    growth_rate = dict()
    lag_phase = dict()
    mean_exp_dGFPdt = dict()
    mean_sta_dGFPdt = dict()
    dGFPdt_divide_OD_max = dict()
    blank_od = 0.04
    pre_max_v = 0.7
    import matplotlib.pyplot as plt
    import numpy as np
    t = t_k
    for l in range(m_OD[:,detectedColumn].shape[1]):
        fn = well_name[detectedColumn[l]]
        out_figer_Name = os.path.join(figure_folder_path, preName +'-'+ fn)
        
        if len(filtered_name):
            if fn not in filtered_name:
                continue
        od = m_OD[:,detectedColumn][:,l]
        od_array = od - np.min(blank_od)
        od_array = np.squeeze(np.asarray(od_array))
        sOD = smooth(od_array)    
        sOD_log = np.log(sOD)
        
        gfp = m_GFP[:,detectedColumn][:,l]
        gfp_array = np.squeeze(np.asarray(gfp))
        sGFP = smooth(gfp_array)

        #---compute lagphase using pre-max-v ---#
        coef_OD, dt_pad = computeDerivetive_od(t, sOD)
        ind_max_dODdt = np.argmax(coef_OD)
        OD_exp = sOD[ind_max_dODdt]
        time_max_dODdt = dt_pad[ind_max_dODdt]
        #t_exp - t0 - ln(OD_exp/OD_t0)/mu_max
        start_positive = getPostiveStart(sOD)
        if start_positive > 0:
            print fn,'found negative od ', start_positive    
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/pre_max_v
        m = t < lag_phase_pre
        start_ind = np.sum(m)

        #---detect the early death pattern---#
        early_death = False
        if sOD[0] > sOD[len(sOD)-1]:
            early_death = True
            #print fn, 'Early death pattern detected'
        if early_death:
            sOD_reverse = sOD[::-1]
            sOD = sOD_reverse
            coef_OD, dt_pad = computeDerivetive_od(t, sOD)
            
            ind_max_dODdt = np.argmax(coef_OD)
            od_max_coef = coef_OD[ind_max_dODdt]
            OD_exp = sOD[ind_max_dODdt]
            time_max_dODdt = dt_pad[ind_max_dODdt]
            # t_exp - t0 - ln(OD_exp/OD_t0)/mu_max
            start_positive = getPostiveStart(sOD)
            if start_positive > 0:
                print fn,'found negative od ', start_positive  
            
            lag_phase_reversed = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/od_max_coef
            # this is an estimated lag_phase
            lag_phase_pre = t[len(t)-1]-lag_phase_reversed
            lag_ravel = np.ravel(lag_phase_pre)
            lag_phase[fn] = lag_ravel[0]
            mean_exp_dGFPdt[fn] = 0.0
            mean_sta_dGFPdt[fn] = 0.0
            dGFPdt_divide_OD_max[fn] = 0.0
            #print type(lag_phase[fn])

            x = t
            y = sOD[::-1]
            plt.title(names_array[l] + '-'+fn)
            plt.plot(x,y,label='OD-Early-Death-Pattern')    
            plt.axvline(x=lag_phase[fn],linestyle=':',color='purple')    
            plt.xlabel('time')
            plt.legend()
            plt.savefig(out_figer_Name+'-Early-Death.png')
            plt.close()
            growth_rate[fn] = 0    
            
            plt.title(names_array[l] + '-'+fn)
            plt.plot(x,y,label='OD-Early-Death-Pattern')    
            plt.axvline(x=lag_phase[fn],linestyle=':',color='purple')    
            plt.xlabel('time')
            plt.legend()                
            growth_rate[fn] = 0    
            plt.savefig(out_figer_Name+'-Early-Death.png')
            plt.close()
            continue
        
        if sOD[len(sOD)-1] < 0.1:
            print fn, 'Flat line (no growth, no lagpahse)'
            x = t
            plt.title(names_array[l] + '-'+fn)
            plt.plot(x,sOD,label='OD-NoGrowth')    
            plt.xlabel('time')
            plt.ylim(top=0.3)
            plt.legend()
            plt.savefig(out_figer_Name+'-NoGrowth.png')
            growth_rate[fn] = 0
            lag_phase[fn] = 0
            mean_exp_dGFPdt[fn] = 0.0
            mean_sta_dGFPdt[fn] = 0.0
            dGFPdt_divide_OD_max[fn] = 0.0
            continue

        #sGFP = sOD
        dGFPdt_divide_OD, coef_OD, coef_GFP, dt_pad= computeDerivetive(t,sOD,sGFP)
        #time_max_od,time_stop,time_left,time_right,time_sta_left,time_sta_right,left_side_exp,right_side_exp,left_side_sta,right_side_sta = getMaxLeftRight(coef_OD,sOD_log,dt_pad,start_ind)
        time_max_od,time_left, time_right,left_side_exp,right_side_exp,small_t,tan = getTimepoints(coef_OD,sOD_log,dt_pad,start_ind)
        
        sta_index = getStationaryTime(sOD)
        t_sta = dt_pad[sta_index]
        
        ind = np.argmax(coef_OD)        
        max_v = coef_OD[ind]    
        growth_rate[fn] = max_v
        
        mean_exp_dGFPdt[fn] = np.mean(dGFPdt_divide_OD[left_side_exp:right_side_exp])
        mean_sta_dGFPdt[fn] = np.mean(dGFPdt_divide_OD[sta_index:])
        ind = np.argmax(dGFPdt_divide_OD)
        dGFPdt_divide_OD_max[fn] = dGFPdt_divide_OD[ind]
        time_max_dGFPdtOD = dt_pad[ind]

        
        # Update lag_phase using growth rate in an estimated window
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/max_v
        lag_phase[fn] = lag_phase_pre
        x = t
        y = sOD_log
        plt.plot(x,y,small_t,tan,'--r',label='log(OD)')
        plt.title(names_array[l] + '-'+fn)
        plt.xlabel("time")        
        w = coef_OD
        plt.plot(x, w,label='dOD/dt/OD') # can c                
        plt.axvline(x=time_max_od,linestyle=':',color='green')
        plt.legend()
        plt.savefig(out_figer_Name+'-dODdtOD.png')
        print out_figer_Name+'-dODdtOD.png'
        #plt.show()
        plt.close()
        
        
        #plt.axvline(x=time_max_dODdt,linestyle=':',color='blue')    
        #plt.axvline(x=time_max_od,linestyle=':',color='green')
        plt.plot(x,sOD,label='OD')            
        plt.axvline(x=time_left,linestyle=':',color='red')
        plt.axvline(x=time_max_od,linestyle=':',color='green')
        plt.axvline(x=t_sta,linestyle=':',color='yellow')
        plt.axvline(x=time_right,linestyle=':',color='red')
        plt.axvline(x=lag_phase[fn],linestyle=':',color='purple')    
        
        plt.xlabel('time')
        plt.title(names_array[l] + '-'+fn)        
        plt.legend()
        plt.savefig(out_figer_Name+'-OD.png')
        #plt.show()
        plt.close()
        
        
        z = sGFP
        z2 = coef_GFP
        plt.plot(x,z, label='GFP')
        plt.plot(x,z2,label='dGFP/dt')
        plt.legend()
        plt.xlabel('time')
        plt.savefig(out_figer_Name+'-GFP-dGFPdt.png')
        plt.close()
        
        plt.plot(x,z,label='GFP')
        plt.plot(x,dGFPdt_divide_OD,label='dGFP/dt/OD')
        plt.axvline(x=time_max_dGFPdtOD,linestyle=':',color='purple')
        plt.axvline(x=t_sta,linestyle=':',color='yellow')
        plt.legend()
        plt.xlabel('time')
        plt.savefig(out_figer_Name+'-dGFPdtOD.png')
        plt.close()
    
    import pandas as pd
    df = pd.DataFrame.from_dict([growth_rate,lag_phase,mean_exp_dGFPdt,mean_sta_dGFPdt,dGFPdt_divide_OD_max])
    df = df.transpose()
    df2 = df.reindex(well_name)
    df2.columns = ['Growth Rate', 'Lag Phase','mean_exp_dGFPdt','mean_sta_dGFPdt','dGFPdt_divide_OD_max']
    df2.to_csv(outName_Stat)


In [9]:
import numpy as np
m_data = dict()
time = dict()
import os
for fn in all_data:
#if True:
    print fn
    folder_path = os.path.splitext(fn)[0]
    out_name = 'RES'
    m_od = all_data[fn]['od']
    time_od = all_data[fn]['time_od']
    m_gfp = all_data[fn]['gfp']
    getStats(m_od,time_od,m_gfp, folder_path)

./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1.csv
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-A4-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-A5-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-A6-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-A10-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-A11-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-A12-dODdtOD.png
stationa

stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-G1-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-G2-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-G3-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-G4-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-G5-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/FIG/20190408 - R97 (2) - SparkControl magellan Sheet 1-G6-dODdtOD.png
stationary phase found
./dataset/20190408 - R97 (2) - SparkControl magellan 

stationary phase found
./dataset/R98 version 2 - SparkControl magellan Sheet 1/FIG/R98 version 2 - SparkControl magellan Sheet 1-D11-dODdtOD.png
stationary phase found
./dataset/R98 version 2 - SparkControl magellan Sheet 1/FIG/R98 version 2 - SparkControl magellan Sheet 1-D12-dODdtOD.png
stationary phase found
./dataset/R98 version 2 - SparkControl magellan Sheet 1/FIG/R98 version 2 - SparkControl magellan Sheet 1-E1-dODdtOD.png
stationary phase found
./dataset/R98 version 2 - SparkControl magellan Sheet 1/FIG/R98 version 2 - SparkControl magellan Sheet 1-E2-dODdtOD.png
stationary phase found
./dataset/R98 version 2 - SparkControl magellan Sheet 1/FIG/R98 version 2 - SparkControl magellan Sheet 1-E3-dODdtOD.png
stationary phase found
./dataset/R98 version 2 - SparkControl magellan Sheet 1/FIG/R98 version 2 - SparkControl magellan Sheet 1-E4-dODdtOD.png
stationary phase found
./dataset/R98 version 2 - SparkControl magellan Sheet 1/FIG/R98 version 2 - SparkControl magellan Sheet 1-E5-dO

stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-B4-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-B5-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-B6-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-B7-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-B8-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlack

stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-F6-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-F7-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-F8-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-F9-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92-F10-dODdtOD.png
stationary phase found
./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/FIG/NB_OD600_meGFP_480_520_FlatBlac